In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from geopy.geocoders import Nominatim
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
color = sns.color_palette()
pd.options.mode.chained_assignment = None  # default='warn'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv("experiment_with_lat_and_lng.csv") 

In [3]:
df.head(2)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AS OF FINAL ROLL 18/19,BLOCK,LOT,BUILDING CLASS AS OF FINAL ROLL 18/19,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,AREA,FINAL_ADDRESS,lat,lng
0,3,FLATBUSH-LEFFERTS GARDEN,01 ONE FAMILY DWELLINGS,1,5050,2,S0,625 ROGERS AVENUE,11225.0,1.0,2.0,3.0,3700.0,2128.0,1905.0,1,S0,1925000,2019-02-05,brooklyn,625 ROGERS AVENUE NYC,40.656321,-73.952859
1,4,HOWARD BEACH,01 ONE FAMILY DWELLINGS,1,14069,139,A1,85-11 164TH AVENUE,11414.0,1.0,0.0,1.0,4000.0,2192.0,1965.0,1,A1,730000,2019-07-02,queens,85-11 164TH AVENUE NYC,40.650046,-73.844749


In [5]:
example1 = "api-key_geocoding.txt"
file1 = open(example1, "r")

In [6]:
API_KEY = file1.read()

In [7]:
import json
import urllib.request

In [8]:
URL_near_by_search = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [9]:
URL_find_place_from_text = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

In [10]:
def getplace(lat, lng, radius):
    locationbias = f"circle:{radius}@{lat},{lng}"
    # Join the parts of the URL together into one string.
    params = urllib.parse.urlencode(
         {
            "input": "Best Western Plus Arena Hotel",
            "inputtype": "textquery",
            "fields": "rating,price_level,user_ratings_total,formatted_address,name",
            "locationbias": locationbias,
            "key": API_KEY,
        }
    )
    url = f"{URL_find_place_from_text}?{params}"
    # result = json.load(urllib.request.urlopen(url))

    # if result["status"] in ["OK", "ZERO_RESULTS"]:
    #     return result["results"]

    # raise Exception(result["error_message"])

In [11]:
results = getplace(40.650046	, -73.94875689999999, 5000)

In [12]:
df.shape[0]

98

In [13]:
URL_text_search = "https://maps.googleapis.com/maps/api/place/textsearch/json"

In [14]:
def get_near_by_search(lat, lng, radius):
    location = f"{lat},{lng}"
    radius = radius
    # Join the parts of the URL together into one string.
    params = urllib.parse.urlencode(
         {
          "query": "cafe",
          "opennow": "true",
          "location": location,
          "radius": radius,
          "type": "cafe",
          "key": API_KEY,
        }
    )
    url = f"{URL_near_by_search}?{params}"
    result = json.load(urllib.request.urlopen(url))

    # if result["status"] in ["OK", "ZERO_RESULTS"]:
    #     return result["results"]

    # raise Exception(result["error_message"])

In [16]:
results = get_near_by_search(40.650046, -73.8447489, 1000)

In [ ]:
len(results)

In [ ]:
df_5 = df.head(5)

In [ ]:
ans = []
for index,row in df_5.iterrows():
    lat, lng = row['lat'], row['lng']
    results = get_near_by_search(lat, lng)
    ans.append(len(results))

In [ ]:
print(ans)

[20, 2, 3, 5, 20]


In [ ]:
df_5['cafe_1000_meters'] = ans

In [ ]:
df_5

In [ ]:
ans = []
for index,row in df.iterrows():
    lat, lng = row['lat'], row['lng']
    results = get_near_by_search(lat, lng)
    ans.append(len(results))

In [ ]:
df['num_cafe_1000_meters'] = ans

In [ ]:
df

In [ ]:
df.drop(columns=['cafe_1000_meters'])

In [ ]:
ans = []
for index,row in df_5.iterrows():
    lat, lng = row['lat'], row['lng']
    results = get_near_by_search(lat, lng, 500)
    ans.append(len(results))

In [ ]:
print(ans)

[8, 0, 2, 1, 4]


In [ ]:
ans = []
for index,row in df.iterrows():
    lat, lng = row['lat'], row['lng']
    results = get_near_by_search(lat, lng, 500)
    ans.append(len(results))

In [ ]:
df['num_cafe_500_meters'] = ans

In [ ]:
df.head(20)

In [ ]:
df = df.drop(columns=['cafe_1000_meters'])

In [ ]:
df

In [ ]:
df.to_csv('experiment_with_lat_and_lng_and_num_cofe.csv', index=False)